In [59]:
import sys
import os

sys.path.append("../Pre_processing")

from Data_Preparation_Library import *
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

image_size=128

In [60]:
#selectedBatches=["6","7","8","9","10","11","12","13","14","15","16","17","18","19","20"]
maxBatchId = 5
selectedBatches=[str(i) for i in range(maxBatchId)]

extra_folder=str(image_size)+"_"+str(image_size)
batch_data_object = []
for i in selectedBatches:
    with open(os.path.join(temp_path,extra_folder,'full_data_object_' + i + '.p'), 'rb') as handle:
        batch_data_object+=pickle.load(handle)

data_train = batch_data_object[0:int(len(batch_data_object)*2/3)]


In [61]:
batch_data_test_object = []
for i in range(8):
    with open(os.path.join(temp_path,extra_folder,'test_data_object_' + str(i) + '.p'), 'rb') as handle:
        batch_data_test_object+=pickle.load(handle)
        
data_test  = batch_data_test_object

In [62]:
def flux(d):
  # TODO: get flux_i from model
    #if d.i_image != None:
    #    flux_i = sum(sum(d.i_image))
    #else:
    #    flux_i = 0
    flux_g = sum(sum(d.g_image_resized))
    #return flux_i+flux_g, flux_g-flux_i
    return flux_g

In [63]:
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation
from keras.layers.core import Dropout

In [64]:
len(batch_data_test_object),batch_data_test_object[0].g_image_resized_reshaped.shape

(8447, (128, 128, 1))

In [66]:
version="v1"
res_folder="results_"+str(image_size)+"_on_"+str(image_size)
with open(os.path.join(os.path.join(output_path,res_folder),"encoder_results_train_"+version+"__all"),'rb') as handle:
    encoded_imgs1=pickle.load(handle)
    encoded_imgs_reshaped_train = [e.reshape(-1,1) for e in encoded_imgs1]

model_encoder=load_model(os.path.join(os.path.join(output_path,res_folder),"encoder_model_"+version+"__all"))

def get_g_Data_for_Autoencoder(data_object):
    X_list=[(data_object[index].g_image_resized_reshaped)/data_object[index].g_image_resized_reshaped.max() for index in range(len(data_object))]
    return np.asarray(X_list),np.asarray(X_list)

X_test_auto_encoder,y_test_auto_encoder=get_g_Data_for_Autoencoder(batch_data_test_object)

encoded_imgs2=model_encoder.predict(X_test_auto_encoder)
encoded_imgs_reshaped_test = [e.reshape(-1,1) for e in encoded_imgs2]


In [67]:
# accessible in the class: size of image, maximum of image, normalization factor of g + fitted value of normalization factor for i
# define a function to get the training and testing dataset

def obtain_train_data():
    train_features = []
    train_labels = []
    for i in range(len(data_train)):
        if data_train[i].logMstar!=-99:
            e = encoded_imgs_reshaped_train[i]
            d = data_train[i]
            #f1,f2 = flux(d)
            f = flux(d)
            size_1=d.g_image.shape[0]
            size_2=d.g_image.shape[1]
            img_max=d.g_image.max()
            dist=d.Distance
            train_features.append(np.append(e,np.array([f,size_1,size_2,img_max,dist])))
            #train_features.append(np.append(e,np.array([f])))
            train_labels.append(d.logMstar)
    return train_features, train_labels

train_features = np.asarray(obtain_train_data()[0])
train_labels = np.asarray(obtain_train_data()[1])

print(train_features.shape, train_labels.shape)

(3277, 4101) (3277,)


In [68]:
def obtain_test_data():
    test_features = []
    test_labels = []
    for j in range(len(data_test)):
        if data_test[j].logMstar!=-99:
            e = encoded_imgs_reshaped_test[j]
            d = data_test[j]

        #f1,f2 = flux(d)
            f = flux(d)
            size_1=d.g_image.shape[0]
            size_2=d.g_image.shape[1]
            img_max=d.g_image.max()
            dist=d.Distance
            test_features.append(np.append(e,np.array([f,size_1,size_2,img_max,dist])))
            test_labels.append(d.logMstar)

    return test_features, test_labels

test_features = np.asarray(obtain_test_data()[0])
test_labels = np.asarray(obtain_test_data()[1])

print(test_features.shape, test_labels.shape)

(8447, 4101) (8447,)


In [69]:
# ## transform features
# train_features = StandardScaler().fit_transform(train_features)
# test_features = StandardScaler().transform(test_features)
# #features = MinMaxScaler().fit_transform(features)

X_scaler = StandardScaler()
train_features = X_scaler.fit_transform(train_features)
test_features = X_scaler.transform(test_features)


In [70]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers.core import Dropout

In [71]:
model = Sequential()
model.add(Dense(64, activation='sigmoid', input_dim=train_features.shape[1]))
model.add(Dropout(0.1))
model.add(Dense(32, activation='sigmoid'))
model.add(Dropout(0.1))
model.add(Dense(1))
model.compile(optimizer='Adadelta',
              loss='mse',
              metrics=['mse'])

In [72]:
model.fit(train_features, train_labels, epochs=20, batch_size=32,shuffle=True,
                validation_data=(test_features, test_labels))

Train on 3277 samples, validate on 8447 samples
Epoch 1/20
3277/3277 [==============================] - 1s - loss: 71.9788 - mean_squared_error: 71.9788 - val_loss: 12.7115 - val_mean_squared_error: 12.7115
Epoch 2/20
3277/3277 [==============================] - 0s - loss: 37.1849 - mean_squared_error: 37.1849 - val_loss: 29.5600 - val_mean_squared_error: 29.5600
Epoch 3/20
3277/3277 [==============================] - 0s - loss: 19.8559 - mean_squared_error: 19.8559 - val_loss: 47.3384 - val_mean_squared_error: 47.3384
Epoch 4/20
3277/3277 [==============================] - 0s - loss: 10.2960 - mean_squared_error: 10.2960 - val_loss: 64.5537 - val_mean_squared_error: 64.5537
Epoch 5/20
3277/3277 [==============================] - 0s - loss: 4.7746 - mean_squared_error: 4.7746 - val_loss: 82.1469 - val_mean_squared_error: 82.1469
Epoch 6/20
3277/3277 [==============================] - 0s - loss: 2.0995 - mean_squared_error: 2.0995 - val_loss: 95.2205 - val_mean_squared_error: 95.2205
Ep

In [73]:
from sklearn.metrics import mean_squared_error

test_predict = []
test_data_sdss_id=[]
for i in range(len(batch_data_test_object)):

    test_predict.append(model.predict(test_features[i].reshape(1,-1))[0][0])
    test_data_sdss_id.append(batch_data_test_object[i].SDSS_ID)
    
print(len(test_predict))



8447


In [77]:
df=pd.DataFrame({"pssid":test_data_sdss_id,"mass":test_predict})
df
df[["pssid","mass"]].to_csv(os.path.join(output_path,"sub_"+version+"_"+str(image_size)+".csv"), index=False)